# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
print(len(city_data_df))
# Display sample data
city_data_df.head()

560


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,east london,-33.0153,27.9116,19.78,83,94,3.72,ZA,1667526336
1,1,kapaa,22.0752,-159.3190,28.18,76,75,6.69,US,1667526336
2,2,port alfred,-33.5906,26.8910,19.29,83,63,7.39,ZA,1667526337
3,3,nantucket,41.2835,-70.0995,13.89,88,0,3.60,US,1667526337
4,4,kungurtug,50.5994,97.5228,-7.52,76,100,0.59,RU,1667526337


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    s='Humidity',
    scale= .7)

# Display the map
# YOUR CODE HERE
plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [59]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]

print(len(ideal_df))
# Drop any rows with null values
# YOUR CODE HERE
ideal_df.dropna()
ideal_df['Country'].value_counts()

# Display sample data
# YOUR CODE HERE
plot_2 = ideal_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    s='Humidity',
    scale= .7)
plot_2

14


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 3: Create a new DataFrame called `hotel_df`.

In [60]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()
hotel_df['Hotel Name']=''
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name']=''
# Display sample data
print(hotel_df.head())
plot_3 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    s='Humidity',
    scale= .7)
plot_3
# YOUR CODE HERE

             City Country      Lat       Lng  Humidity Hotel Name
240     bonavista      CA  48.6499  -53.1147        63           
248       te anau      NZ -45.4167  167.7167        68           
301        torbay      CA  47.6666  -52.7314        64           
321  machacamarca      BO -18.1667  -67.0333        23           
414         jumla      NP  29.2747   82.1838        38           


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
# Set parameters to search for a hotel
# YOUR CODE HERE
radius = 10000
params = {'categories':'accommodation.hotel',\
#          'conditions':'internet_access',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
#     print(json.dumps(name_address, indent=4,sort_keys=True))
#     print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
#         # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df

Starting hotel search
bonavista - nearest hotel: Harbour Quarters Inn
te anau - nearest hotel: Lakeside Motels & Apartments
torbay - nearest hotel: Hampton
machacamarca - nearest hotel: No hotel found
jumla - nearest hotel: Amar Sandesh Guesthouse
port hawkesbury - nearest hotel: Hearthstone Inn Port Hawkesbury
damavand - nearest hotel: No hotel found
emborion - nearest hotel: No hotel found
sanandaj - nearest hotel: هتل آبیدر
tuatapere - nearest hotel: No hotel found
zaysan - nearest hotel: Рахат қонақүй
nemuro - nearest hotel: イーストハーバーホテル
villazon - nearest hotel: Hotel Ideal
creel - nearest hotel: Hotel Cascada inn


,City,Country,Lat,Lng,Humidity,Hotel Name
240,bonavista,CA,48.6499,-53.1147,63,Harbour Quarters Inn
248,te anau,NZ,-45.4167,167.7167,68,Lakeside Motels & Apartments
301,torbay,CA,47.6666,-52.7314,64,Hampton
321,machacamarca,BO,-18.1667,-67.0333,23,No hotel found
414,jumla,NP,29.2747,82.1838,38,Amar Sandesh Guesthouse
417,port hawkesbury,CA,45.6169,-61.3485,42,Hearthstone Inn Port Hawkesbury
425,damavand,IR,35.7178,52.0650,69,No hotel found
451,emborion,GR,40.4833,21.5500,51,No hotel found
483,sanandaj,IR,35.3144,46.9923,69,هتل آبیدر
487,tuatapere,NZ,-46.1333,167.6833,69,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [62]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
plot_4 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= .7)
# Display the map
# YOUR CODE HERE
plot_4

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)

In [ ]:
# name_address["features"][0]["properties"]["datasource"]["raw"]["name:en"]